In [ ]:
from os import getenv
from dotenv import load_dotenv
from langchain_community.graphs import Neo4jGraph
from langchain.chains import GraphCypherQAChain
from langchain_google_genai import ChatGoogleGenerativeAI

ImportError: cannot import name 'ChatGoogleGenerativeAI' from 'langchain.llms' (c:\Users\Admin\VIP\Máy tính\NCKH\test\.venv\Lib\site-packages\langchain\llms\__init__.py)

#### You will need [Neo4j APOC](https://neo4j.com/docs/apoc/current/installation/) for langchain_community.graph.Neo4jGraph to function.
Follow the instructions in the link and then, in neo4j.conf, set 
`dbms.security.procedures.unrestricted=apoc.*`

In [3]:
load_dotenv()

True

In [6]:
neo4j_url= getenv("NEO4J_URL")
neo4j_user = getenv("NEO4J_USER")
neo4j_password = getenv("NEO4J_PW")
neo4j_database = getenv("NEO4J_DB")

In [7]:
graph = Neo4jGraph(neo4j_url, neo4j_user, neo4j_password, neo4j_database)

In [8]:
print(graph.schema)

Node properties:
Movie {certificate: STRING, rating: FLOAT, run_time: INTEGER, tagline: STRING, title: STRING, year: INTEGER}
Person {name: STRING}
Actor {name: STRING}
Director {name: STRING}
Writer {name: STRING}
Genre {name: STRING}
Relationship properties:

The relationships:
(:Movie)-[:BELONGS_TO]->(:Genre)
(:Person)-[:ACTED_IN]->(:Movie)
(:Person)-[:DIRECTED]->(:Movie)
(:Person)-[:WROTE]->(:Movie)
(:Actor)-[:ACTED_IN]->(:Movie)
(:Actor)-[:WROTE]->(:Movie)
(:Actor)-[:DIRECTED]->(:Movie)
(:Director)-[:DIRECTED]->(:Movie)
(:Director)-[:WROTE]->(:Movie)
(:Director)-[:ACTED_IN]->(:Movie)
(:Writer)-[:DIRECTED]->(:Movie)
(:Writer)-[:WROTE]->(:Movie)
(:Writer)-[:ACTED_IN]->(:Movie)


#### Gemini 

In [9]:
gemini_api = getenv("GEMINI_API")

In [32]:
llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro", google_api_key = gemini_api ,temperature = 0)

chain = GraphCypherQAChain.from_llm(graph=graph, llm=llm, verbose=True,allow_dangerous_requests=True)

### Prompts

- #### Simple prompts.

In [35]:
questions = [
    "Which are the best movies released before 1960?",
    # "Which are the 3 best crime dramas?",
    # "Who is the director with highest average rating for their movies?",
    # "In what movies have Robert DeNiro and Al Pacino together?",
    # "Which director has Brad Pitt worked with the most?",
    # "Best 5 thrillers made before 2000."
]

for question in questions:
    print("\n\n" + question)
    print(chain.invoke(question)["result"])



Which are the best movies released before 1960?


> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (m:Movie)
WHERE m.year < 1960
RETURN m
ORDER BY m.rating DESC
LIMIT 10

Full Context:
[{'m': {'run_time': 96, 'year': 1957, 'certificate': 'Approved', 'rating': 9.0, 'tagline': 'Life Is In Their Hands -- Death Is On Their Minds!', 'title': '12 Angry Men'}}, {'m': {'run_time': 130, 'year': 1946, 'certificate': 'PG', 'rating': 8.6, 'tagline': 'Frank Capra\'s..."It\'s a Wonderful Life".', 'title': "It's a Wonderful Life"}}, {'m': {'run_time': 207, 'year': 1954, 'certificate': 'Not Rated', 'rating': 8.6, 'tagline': 'Will Take Its Place With the Seven Greatest Films of All Time!', 'title': 'Seven Samurai'}}, {'m': {'run_time': 112, 'year': 1954, 'certificate': 'PG', 'rating': 8.5, 'tagline': 'The Essential Hitchcock', 'title': 'Rear Window'}}, {'m': {'run_time': 87, 'year': 1936, 'certificate': 'G', 'rating': 8.5, 'tagline': 'He stands alone as the greatest entertain

- #### Some prompts need clarification. In this case, writers, directors  and actors who have worked together need to be specified as different people.

In [37]:
questions = [
    "Which actor and director have worked together the most?",
    # "Which writer and director have worked together the most?",

    # "Which actor and director have worked together the most? Remember they cannot be the same person.",
    # "Which writer and director have worked together the most? Remember they cannot be the same person."
]

for question in questions:
    print("\n\n" + question)
    print(chain.invoke(question)["result"])



Which actor and director have worked together the most?


> Entering new GraphCypherQAChain chain...


Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..


Generated Cypher:
cypher
MATCH (a:Actor)-[:ACTED_IN]->(m:Movie)<-[:DIRECTED]-(d:Director)
RETURN a.name, d.name, COUNT(m) AS collaborations
ORDER BY collaborations DESC
LIMIT 1

Full Context:
[{'a.name': 'Charles Chaplin', 'd.name': 'Charles Chaplin', 'collaborations': 5}]


Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..



> Finished chain.
Charles Chaplin has collaborated with himself (as director) 5 times.


- #### Sometimes the LLM generates proper Cypher queries, which return the proper results, but the LLM is unable to infer the required information from these results.
- #### Adding another line that motivates it to understand helps in these cases.

In [14]:
questions = [
    "Who is the actor who has starred in most of the best rated movies.",
    
    "Who is the actor who has starred in most of the best rated movies." \
    "You know the answer. Just parse the result from the Cypher query, understand it, then tell me the actor."
]

for question in questions:
    print("\n" + question)
    print(chain.invoke(question)["result"])


Who is the actor who has starred in most of the best rated movies.


> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (a:Actor)-[:ACTED_IN]->(m:Movie)
WHERE m.rating > 4.0
RETURN a.name, COUNT(m) AS num_movies
ORDER BY num_movies DESC
LIMIT 1;

Full Context:
[{'a.name': 'Robert De Niro', 'num_movies': 9}]

> Finished chain.
I don't know the answer to this question.

Who is the actor who has starred in most of the best rated movies.You know the answer. Just parse the result from the Cypher query, understand it, then tell me the actor.


> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (a:Actor)-[:ACTED_IN]->(m:Movie)
WHERE m.rating > 4.0
RETURN a.name AS actor_name, COUNT(m) AS num_movies
ORDER BY num_movies DESC
LIMIT 1;

Full Context:
[{'actor_name': 'Robert De Niro', 'num_movies': 9}]

> Finished chain.
Robert De Niro


### Enter your own prompts -

In [21]:
while True:
    question = input("\n\nEnter your question (Type 'EXIT' to exit) : ")
    
    if question == "EXIT":
        print("Exiting")
        break
    else:
        print("\n" + question)
        print(chain.invoke(question)["result"])



Enter your question (Type 'EXIT' to exit) :  Top thrillers made before 2000?



Top thrillers made before 2000?


> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (m:Movie)-[:BELONGS_TO]->(g:Genre)
WHERE g.name = 'Thriller' AND m.year < 2000
RETURN m.title AS MovieTitle, m.year AS YearReleased, g.name AS Genre
ORDER BY m.rating DESC
LIMIT 10;

Full Context:
[{'MovieTitle': 'The Silence of the Lambs', 'YearReleased': 1991, 'Genre': 'Thriller'}, {'MovieTitle': 'Psycho', 'YearReleased': 1960, 'Genre': 'Thriller'}, {'MovieTitle': 'Rear Window', 'YearReleased': 1954, 'Genre': 'Thriller'}, {'MovieTitle': 'Come and See', 'YearReleased': 1985, 'Genre': 'Thriller'}, {'MovieTitle': 'Vertigo', 'YearReleased': 1958, 'Genre': 'Thriller'}, {'MovieTitle': 'Reservoir Dogs', 'YearReleased': 1992, 'Genre': 'Thriller'}, {'MovieTitle': 'M', 'YearReleased': 1931, 'Genre': 'Thriller'}, {'MovieTitle': 'Dial M for Murder', 'YearReleased': 1954, 'Genre': 'Thriller'}, {'MovieTitle': 'The Wages of Fear', 'YearReleased': 1953, 'Genre': 'Thriller'}, {'MovieTitle': 'T



Enter your question (Type 'EXIT' to exit) :  Top 5 thrillers less than 120 minutes long?



Top 5 thrillers less than 120 minutes long?


> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (m:Movie)-[:BELONGS_TO]->(g:Genre)
WHERE g.name = 'Thriller' AND m.run_time < 120
RETURN m.title, m.run_time
ORDER BY m.run_time
LIMIT 5;

Full Context:
[{'m.title': 'Persona', 'm.run_time': 83}, {'m.title': 'The Third Man', 'm.run_time': 93}, {'m.title': 'Fargo', 'm.run_time': 98}, {'m.title': 'M', 'm.run_time': 99}, {'m.title': 'Reservoir Dogs', 'm.run_time': 99}]

> Finished chain.
1. Persona (83 minutes)
2. The Third Man (93 minutes)
3. Fargo (98 minutes)
4. M (99 minutes)
5. Reservoir Dogs (99 minutes)




Enter your question (Type 'EXIT' to exit) :  Best Akira Kurosawa movie?



Best Akira Kurosawa movie?


> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (m:Movie)<-[:DIRECTED]-(d:Director {name: "Akira Kurosawa"})
RETURN m.title AS BestMovie
ORDER BY m.rating DESC
LIMIT 1;

Full Context:
[{'BestMovie': 'Seven Samurai'}]

> Finished chain.
Seven Samurai




Enter your question (Type 'EXIT' to exit) :  Who acted in Seven Samurai?



Who acted in Seven Samurai?


> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Movie {title: "Seven Samurai"})-[:ACTED_IN]-(a:Actor) RETURN a.name
Full Context:
[{'a.name': 'Keiko Tsushima'}, {'a.name': 'Takashi Shimura'}, {'a.name': 'Toshirô Mifune'}]

> Finished chain.
Keiko Tsushima, Takashi Shimura, Toshirô Mifune acted in Seven Samurai.




Enter your question (Type 'EXIT' to exit) :  EXIT


Exiting
